In [3]:
import requests
from bs4 import BeautifulSoup
from newspaper import Article
from tqdm import tqdm
import pandas as pd


In [4]:
def repetetive(links, url):
    for link in links:
        if link.a['href'] in url:
            return True
    return False

In [5]:
def scrap_year(year: int):
    page = 6
    scraped_data = []
    url_list = []
    
    while True:
       page -= 1
    
       page_url = f"https://www.mojnews.com/newsstudios/archive/?curp=1&order=order_time&offset%5Bseen%5D=434800&offset%5Btime%5D={year}-06-02+{page}%3A30%3A58&dir=0#gsc.tab=0"
        
       html = requests.get(page_url).text
    
       soup = BeautifulSoup(html, features='lxml')
       
       links = soup.find_all('h2', {"itemprop":"name" , "class":"title"})
        
       if page == 0:
            break
            
       for link in tqdm(links):
            news_url = 'https://www.mojnews.com/' + link.a['href']
            url_list.append(link.a['href'])
            try:
                article = Article(news_url)
                article.download()
                article.parse()
                scraped_data.append({'url':news_url, 'text':article.text, 'title':article.title})
            except:
                print(f'faile :{news_url}')
    print(len(scraped_data))
    df=pd.DataFrame(scraped_data)
    df.to_csv(f'mojnews-{year}.csv')
            
if __name__ == '__main__':
    scrap_year(2022)
    

100%|██████████| 15/15 [00:03<00:00,  4.09it/s]


75


In [6]:
data = pd.read_csv('mojnews-2022.csv')
print(data['text'][0])

به گزارش خبرگزاری موج، سرهنگ رامین پاشایی معاون فرهنگی اجتماعی پلیس فتا انتظامی کشور اظهار داشت: بر اساس تجزیه و تحلیل پرونده‌های قضائی واصله به پلیس فتا و تجربه عملی از ادوار گذشته، همزمان با نزدیک شدن به ایام پایانی هفته و یا تعطیلات رسمی؛ نرخ وقوع ارتکاب جرایم سایبری افزایش می‌یابد که طی اقدام پیشگیرانه تمامی رده‌های پلیس فتا، استان و شهرستان‌ها آمادگی لازم در خصوص رصد و برخورد با هرگونه پدیده مجرمانه و پاسخگویی به درخواست‌های حضوری و تلفنی هموطنان مهیا است.

وی ادامه داد: مجرمان سایبری با سوءاستفاده از شرایط پیش آمده با فرض محدودیت‌های دسترسی به خدمات بانکی، قضائی و انتظامی مترصد کلاهبرداری از هموطنان با شگردهای مختلف "فیشینگ و یا ثبت آنلاین آگهی‌های جعلی فروش و ارائه کالا و خدمات در سایت‌های فروشگاهی" هستند.

پاشایی افزود: عمدتاً کلاهبرداران با ارسال پیامک حاوی لینک‌های آلوده با موضوعاتی از قبیل ثبت نام یا حذف یارانه معیشتی، ابلاغیه‌های دروغین سامانه قضائی کشور و یا دریافت بیعانه فروش کالا و خدمات مترصد کلاهبرداری از هموطنان هستند.

معاون فرهنگی اجتماعی پلیس فتا انتظامی کشور خاطرن

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
import json
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import numpy as np

In [10]:
docs = pd.read_csv('mojnews-2022.csv')

In [11]:
words = []
for d in docs['text'].fillna(""):
  words.extend(d.split())
vocab = set(words)
print(f"Unique words: {vocab}")
print(f"Number of unique words: {len(vocab)}")


Unique words: {'طنبی،', 'می\u200cرسد', 'بازپرداخت', 'بعضی', 'گرم', 'خاقانی،', '10', 'دروغین', 'ترانسفر', 'حداقل', 'حضرت', 'موضوع', 'واصله', 'بروز', 'دیدگاه', 'سرپرست', '52,365', 'ماه', 'یکم', 'حرم', 'گفت:', 'خبری', 'تشرف', 'درآمدزایی', 'گرامیداشت', 'جهت', 'معروف', 'زائران', 'علامه', 'موقت', 'داشتند،', 'خبرگان', 'سازمان،', 'اصفهانی', 'می', 'آمریکا', 'ادامه،', 'عملکرد', 'مراجعه', 'دهه', 'خرید', 'نکرده', 'طرح', 'نظام', '(۱۳', 'عبور', 'جدید', 'بسیاری', 'امتحان', 'سرهنگ', 'توزیع', 'خطه', 'پیشرفت', 'بود.', 'خاطرنشان', 'شهر', 'بازدارندگی', 'گذشته،', 'تعامل', 'میزان', 'طارمی', 'تعیین', 'یا', 'کردن', 'پیچیده', 'شدن', 'همواره', 'برخورد', 'بیمه', 'دولت', 'اخلاص', 'حفاظت', 'اتفاق', 'تقدیر', 'باشند،', 'ساز', 'فجرسپاسی', 'استان', 'تجدید', 'قرارگاه', 'بی\u200cبرنامگی', 'مهمانسراهای', 'معیشتی،', 'عیار', 'اقامت', 'اهرم', 'طوسی', 'آمد', 'کسب', 'مانده', 'سرپوشیده', 'فراهم', 'سوی', 'خوشبختانه', 'همزمان', 'نامه', 'بیماری', 'اجرای', 'حالت', 'نشانی', 'در', 'شهادت', 'کد', 'رفت', 'فرماندهی', 'پیگیری', 'داخل', 

In [17]:
vectorizer = TfidfVectorizer(lowercase=True)
vectorizer.fit(docs["text"].fillna(""))
tfidf_docs = vectorizer.fit_transform(docs["text"].fillna(""))

In [23]:
vectorizer.vocabulary_

{'به': 369,
 'گزارش': 1613,
 'خبرگزاری': 595,
 'موج': 1284,
 'سرهنگ': 858,
 'رامین': 737,
 'پاشایی': 1466,
 'معاون': 1234,
 'فرهنگی': 1053,
 'اجتماعی': 96,
 'پلیس': 1488,
 'فتا': 1042,
 'انتظامی': 235,
 'کشور': 1557,
 'اظهار': 166,
 'داشت': 652,
 'بر': 318,
 'اساس': 135,
 'تجزیه': 416,
 'تحلیل': 420,
 'پرونده': 1484,
 'های': 1385,
 'قضائی': 1101,
 'واصله': 1433,
 'تجربه': 415,
 'عملی': 1025,
 'از': 131,
 'ادوار': 110,
 'گذشته': 1596,
 'همزمان': 1404,
 'با': 279,
 'نزدیک': 1325,
 'شدن': 909,
 'ایام': 266,
 'پایانی': 1470,
 'هفته': 1398,
 'یا': 1630,
 'تعطیلات': 449,
 'رسمی': 751,
 'نرخ': 1324,
 'وقوع': 1456,
 'ارتکاب': 119,
 'جرایم': 513,
 'سایبری': 839,
 'افزایش': 176,
 'می': 1299,
 'یابد': 1632,
 'که': 1583,
 'طی': 992,
 'اقدام': 186,
 'پیشگیرانه': 1511,
 'تمامی': 465,
 'رده': 744,
 'استان': 137,
 'شهرستان': 937,
 'ها': 1383,
 'آمادگی': 60,
 'لازم': 1108,
 'در': 662,
 'خصوص': 610,
 'رصد': 756,
 'برخورد': 325,
 'هرگونه': 1391,
 'پدیده': 1474,
 'مجرمانه': 1145,
 'پاسخگویی': 1465,
 'درخو

In [24]:
docs_tfidf = vectorizer.transform(docs)
type(docs_tfidf), docs_tfidf.shape

(scipy.sparse.csr.csr_matrix, (4, 1686))

In [28]:
list(vectorizer.vocabulary_.keys())[:10]
# query
query = 'پلیس فتا'

tfidf_query = vectorizer.transform([query])[0]

In [29]:
# similarities
cosines = []
for d in tqdm(tfidf_docs):
    cosines.append(float(cosine_similarity(d, tfidf_query)))

75it [00:00, 2086.08it/s]


In [30]:
# sorting
k = 10
sorted_ids = np.argsort(cosines)
sorted_ids
for i in range(k):
    cur_id = sorted_ids[-i-1]
    print(cur_id)
    print(docs['text'][cur_id], cosines[cur_id])

0
به گزارش خبرگزاری موج، سرهنگ رامین پاشایی معاون فرهنگی اجتماعی پلیس فتا انتظامی کشور اظهار داشت: بر اساس تجزیه و تحلیل پرونده‌های قضائی واصله به پلیس فتا و تجربه عملی از ادوار گذشته، همزمان با نزدیک شدن به ایام پایانی هفته و یا تعطیلات رسمی؛ نرخ وقوع ارتکاب جرایم سایبری افزایش می‌یابد که طی اقدام پیشگیرانه تمامی رده‌های پلیس فتا، استان و شهرستان‌ها آمادگی لازم در خصوص رصد و برخورد با هرگونه پدیده مجرمانه و پاسخگویی به درخواست‌های حضوری و تلفنی هموطنان مهیا است.

وی ادامه داد: مجرمان سایبری با سوءاستفاده از شرایط پیش آمده با فرض محدودیت‌های دسترسی به خدمات بانکی، قضائی و انتظامی مترصد کلاهبرداری از هموطنان با شگردهای مختلف "فیشینگ و یا ثبت آنلاین آگهی‌های جعلی فروش و ارائه کالا و خدمات در سایت‌های فروشگاهی" هستند.

پاشایی افزود: عمدتاً کلاهبرداران با ارسال پیامک حاوی لینک‌های آلوده با موضوعاتی از قبیل ثبت نام یا حذف یارانه معیشتی، ابلاغیه‌های دروغین سامانه قضائی کشور و یا دریافت بیعانه فروش کالا و خدمات مترصد کلاهبرداری از هموطنان هستند.

معاون فرهنگی اجتماعی پلیس فتا انتظامی کشور خاط